In [45]:

import requests

def login_and_get_cookie(url, username, password):
    login_url = url + "/api/login"
    payload = {
        "captcha": "",
        "username": username,
        "password": password,
        "returnUrl": ""
    }
    
    session = requests.Session()
    response = session.post(login_url, json=payload)
    
    if response.status_code == 200:
        gosso_cookie = session.cookies.get("GOSSOcookie")
        if gosso_cookie:
            print("로그인 성공, 쿠키 저장 완료")
            return gosso_cookie
        else:
            print("쿠키를 찾을 수 없습니다.")
            return None
    else:
        print("로그인 실패")
        return None


login_and_get_cookie()

로그인 실패


In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

base_url = ""
all_posts_content = []

def crawl_table_and_click_posts(url, username, password):
    # Selenium WebDriver 설정
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))  # Chrome 드라이버 사용
    driver.implicitly_wait(5)

    driver.get(url)

    username_input = driver.find_element(By.NAME, "username")
    password_input = driver.find_element(By.NAME, "password")
    username_input.send_keys(username)
    password_input.send_keys(password)

    login_button = driver.find_element(By.ID, "login_submit")
    login_button.click()

    # 페이지 로딩을 기다림
    time.sleep(2)

    driver.get(url)
    time.sleep(2)

    # 안내 모달 Skip
    welcome = driver.find_elements(By.CSS_SELECTOR, ".welcome_skip")
    welcome[0].click()

    # 테이블 내 tr 태그 클릭 (tr 태그에 적절한 클래스나 id를 사용하여 선택자 수정 필요)
    post_elements = driver.find_elements(By.CSS_SELECTOR, "tr.odd")  # tr 요소 선택자

    data_ids = []
    for post_element in post_elements:
        data_id = post_element.get_attribute('data-id')
        if data_id:
            data_ids.append(data_id)

    for data_id in data_ids:
        try:
            # URL에 data_id 추가하여 페이지로 이동
            post_url = f"{url}/post/{data_id}"
            driver.get(post_url)

            # 페이지 로딩 기다림
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".article_header"))
            )

            # 페이지 크롤링
            page_content = driver.page_source 
            soup = BeautifulSoup(page_content, "html.parser")

            # iframe 내부의 title과 content 추출
            title = soup.select_one(".article_header").text.strip()
            print(title)
            # iframe 내부로 전환
            iframe = driver.find_element(By.CSS_SELECTOR, "#boardContentViewer")
            driver.switch_to.frame(iframe)

            # 페이지 크롤링
            page_content = driver.page_source 
            soup = BeautifulSoup(page_content, "html.parser")

            # iframe 내부의 title과 content 추출
            # title = soup.select_one(".article_header").text.strip()
            content = soup.select_one("#content").text.strip()
            print(content)
            all_posts_content.append({"title": title, "content": content})
            print(f"크롤링 완료: {title}")

            # 원래 페이지로 돌아가기
            driver.switch_to.default_content()

        except Exception as e:
            print(f"오류 발생: {e}")

    # 크롤링된 내용 출력
    for post in all_posts_content:
        print(post)

    driver.quit()  # 드라이버 종료

url = base_url + "/app/board/6078"
crawl_table_and_click_posts(url, cookie)


오류 발생: 'NoneType' object has no attribute 'text'
오류 발생: 'NoneType' object has no attribute 'text'
오류 발생: 'NoneType' object has no attribute 'text'
오류 발생: 'NoneType' object has no attribute 'text'
오류 발생: 'NoneType' object has no attribute 'text'
오류 발생: 'NoneType' object has no attribute 'text'
오류 발생: 'NoneType' object has no attribute 'text'
오류 발생: 'NoneType' object has no attribute 'text'
오류 발생: 'NoneType' object has no attribute 'text'
오류 발생: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: unable to send message to renderer
  (Session info: chrome=132.0.6834.83)
Stacktrace:
#0 0x5602fb2a67ca <unknown>
#1 0x5602fad9e2f0 <unknown>
#2 0x5602fad855e7 <unknown>
#3 0x5602fad85565 <unknown>
#4 0x5602fad83d3a <unknown>
#5 0x5602fad84a2f <unknown>
#6 0x5602fada1f55 <unknown>
#7 0x5602fada2891 <unknown>
#8 0x5602fada6988 <unknown>
#9 0x5602fada6a13 <unknown>
#10 0x5602fadedaf1 <unknown>
#11 0x5602fadee251 <unknown>
#12 0x5602fae34054 <unk

In [53]:
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS

documents = [
    Document(page_content=item["content"], metadata={"title": item["title"]})
    for item in all_posts_content
]

embeddings = OpenAIEmbeddings()

index = VectorstoreIndexCreator(
    vectorstore_cls=FAISS,
    embedding=embeddings,
).from_documents(documents)

index.vectorstore.save_local("faiss")



/tmp/ipykernel_93366/654169887.py:12: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [54]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import VectorDBQA
import gradio as gr
import os
import time
import openai
import ingest

chat = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.9, 
                #   streaming=True
                  )

vectorstore = FAISS.load_local(
        folder_path="faiss", 
        embeddings=OpenAIEmbeddings(),
        allow_dangerous_deserialization=True
    )

def create_generator(history):
    user_input = history[-1][0]

    if not isinstance(user_input, str):
        return "업로드가 완료되었습니다."

    similar_docs = vectorstore.similarity_search(user_input, k=3)
    
    context = "\n\n".join([doc.page_content for doc in similar_docs])
    prompt = f"""
    다음은 사용자가 입력한 질문입니다:
    "{user_input}"
    
    그리고 관련 문서입니다:
    "{context}"
    
    질문이 문서와 관련이 있다면 관련 문서를 참조해 대답해주세요.
    """
    
    gpt_response = chat.predict(prompt)
    return gpt_response

def add_text(history, text):
    history = history + [(text, None)]
    return history, gr.update(value="", interactive=False)

def add_file(history, file):
    ingest.pdfToVectorStore(file)
    history = history + [((file.name,), None)]
    return history

def bot(history):
    response = create_generator(history)
    history[-1][1] = response
    return history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(
        [],
        elem_id="chatbot",
        bubble_full_width=False,
        avatar_images=(None, (os.path.join("./avatar.png"))),
    )

    with gr.Row():
        txt = gr.Textbox(
            scale=4,
            show_label=False,
            placeholder="텍스트를 입력하고 엔터를 치거나 이미지를 업로드하세요",
            container=False,
        )
        btn = gr.UploadButton("Upload", file_types=[".pdf"])

    txt_msg = txt.submit(add_text, [chatbot, txt], [chatbot, txt], queue=False).then(
        bot, chatbot, chatbot
    )
    txt_msg.then(lambda: gr.update(interactive=True), None, [txt], queue=False)
    file_msg = btn.upload(add_file, [chatbot, btn], [chatbot], queue=False).then(
        bot, chatbot, chatbot
    )

demo.queue()
demo.launch(server_name='0.0.0.0')

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_93366/2193837278.py:11: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.9,
/home/hoon/.local/lib/python3.10/site-packages/gradio/components/chatbot.py:273: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages

* Running on local URL:  http://0.0.0.0:7860

To create a public link, set `share=True` in `launch()`.


/tmp/ipykernel_93366/2193837278.py:40: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  gpt_response = chat.predict(prompt)


In [13]:
import pymysql

host = '192.168.10.183'
port = 3306
username  ='root'
password = "1q2w3e4r%T"
database_schema ='wikidb'

connection = pymysql.connect(host= host,
                             port=port,
                             user= username,
                             password=password,
                             db=database_schema)

query = 'SELECT * FROM CONTENT WHERE CONTENTID =327681;'
cursor = connection.cursor()
cursor.execute(query)
print(cursor)
for result in cursor:
    print(result)

(327681, 2, 'USERINFO', None, None, 2, None, datetime.datetime(2018, 10, 10, 4, 22, 50), '8a808042665c354601665c368a3b0000', datetime.datetime(2021, 2, 9, 11, 31, 2), '', None, 'current', None, None, None, None, None, None, None, None, None, None, None, None, None, '8a808042665c354601665c368a3b0000')
